In [ ]:
from TradingView import Datafeed
import pandas as pd

Data = Datafeed()
prices_df = pd.DataFrame()

crypto_list = [('BTCUSD', 'COINBASE'),
               ('ETHUSD', 'COINBASE'),
               ('BNBUSD', 'BINANCE'),
               ('ADAUSD', 'BINANCE'),
               ('LUNAUSD', 'BINANCE'),
               ('XRPUSD', 'BINANCE'),
               ('LINKUSD', 'BINANCE'),
               ('CROUSD', 'BITTREX'),
               ('MANAUSD', 'BINANCE'),
               ('FTTUSD', 'FTX')]

for crypto in crypto_list:
    prices_df[crypto[0]] = Data.daily_close(symbol=crypto[0], exchange=crypto[1], 
                                            date_start_str='2020/01/01', date_end_str='2022/01/01')

In [ ]:
import matplotlib.pyplot as plt

for crypto in prices_df.columns:
    plt.figure(figsize = (16, 9))
    plt.plot(prices_df[crypto], label = crypto)
    plt.title(crypto)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
from MethodUtil import cal_return, descriptive_statistics
return_df = cal_return(prices_df).dropna()
descriptive_statistics(return_df)

In [ ]:
logreturn_df = cal_return(prices_df, log=True).dropna()
descriptive_statistics(logreturn_df)

In [ ]:
from MethodUtil import return_spreads
return_spreads_df = return_spreads(logreturn_df)
descriptive_statistics(return_spreads_df)

In [ ]:
from MethodUtil import pair_selection_MSD
MSD_df = pair_selection_MSD(prices_df)

from MethodUtil import cointegration_test
cointegration_df = cointegration_test(prices_df, alpha=0.05)

pd.concat([MSD_df, cointegration_df], axis='columns')

In [ ]:
# selected_list = [('ETHUSD', 'BNBUSD'), ('ADAUSD', 'FTTUSD'), ('BNBUSD', 'FTTUSD'), ('ETHUSD', 'FTTUSD'), ('XRPUSD', 'FTTUSD'), ('ADAUSD', 'XRPUSD')]
selected_list = [('ETHUSD', 'BNBUSD'),('XRPUSD', 'FTTUSD')]

In [ ]:
from MethodUtil import price_normalization
for pair in selected_list:
    prices_norm = price_normalization(prices_df)
    plt.figure(figsize = (16, 9))
    plt.plot(prices_norm[pair[0]], label = pair[0])
    plt.plot(prices_norm[pair[1]], label = pair[1])
    plt.title(f'{pair[0]} v.s. {pair[1]}')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
from MethodUtil import ForecastModel
from arch.univariate import *
model_list = []
for pair in selected_list:
    for vol in [GARCH(1,0,1), GARCH(1,1,1), EGARCH(1,0,1)]:
        for dist in [Normal(), StudentsT(), SkewStudent()]:
            Model = ForecastModel(return_spreads_df, pair=pair)
            Model.modelspec(vol=vol, dist=dist)
            for alpha in [[0.2, 0.8],[0.1, 0.9]]:
                Model.rollingForecasting(window=365, VaR_alpha=[0.1, 0.9])
                model_list.append(Model)

In [ ]:
from MethodUtil import Strategy
dfs = []
for model in model_list:
    strategy = Strategy(prices_df, model)
    tmp_df = strategy.performance(date_start='2021-01-01')
    dfs.append(tmp_df)
pd.concat(dfs)

In [ ]:
strategy = Strategy(prices_df, model_list[0])
plt.figure(figsize = (16, 9))
plt.plot(strategy.spread, label = 'spread')
plt.plot(strategy.lowerVaR, label = 'lower bound')
plt.plot(strategy.upperVaR, label = 'upper bound')
plt.title('')
plt.legend()
plt.grid()
plt.show()

In [ ]:
model_list[0].res.summary()